# Recommendation

In [11]:
import networkx as nx
import random
import pandas as pd

In [42]:
import warnings
warnings.filterwarnings('ignore')

# Import graph

In [43]:
# Path to your GraphML file
graphml_file_path = 'neo4jdata.graphml'

# Read the GraphML file
G = nx.read_graphml(graphml_file_path)

# Warning은 무시해도 상관 없습니다.

In [44]:
# userID 또는 categoryID로 각 node를 찾는 함수입니다.
# 이는 graphDB에서 networkX로 옮겨 오면서 발생하는 ID 차이로 인해 사용합니다.
def find_node_by_attribute(G, attr, value):
    for node, data in G.nodes(data=True):
        if data.get(attr) == value:
            return node
    return None

# Category recommendation

In [45]:
def category_recommender(G, user_id, key_category_id):
    # Step 1: Find the keySupercategory of the keyCategory
    key_category_node = find_node_by_attribute(G, 'categoryID', key_category_id)
    key_supercategory_node = None
    for u, v, d in G.edges(key_category_node, data=True):
        if d.get('label') == 'CATEGORY_BELONGS_TO':
            key_supercategory_node = v
            break

    if not key_supercategory_node:
        raise ValueError("Key supercategory not found")
    
    # Step 2: Find similar users
    target_user_node = find_node_by_attribute(G, 'userID', str(user_id))
    # Similar user가 없는 경우에는 이를 빈 list로 해줘야 한다. 
    # 이렇게 하지 않으면 모든 edges를 다 가져오기 때문에 문제가 생긴다.
    if target_user_node:
        similar_users = [v for u, v in G.edges(target_user_node) if G[u][v].get('label') == 'SIMILAR']
    else:
        similar_users = []
    
    # Initialize recommended_categories
    recommended_categories = {}

    # If there are similar users, proceed with collaborative filtering
    if similar_users:
        for su in similar_users:
            for u, v, d in G.edges(su, data=True):
                if d.get('label') == 'PREFERENCE':
                    if any(d.get('label') == 'CATEGORY_BELONGS_TO' for _, _, d in G.edges(v, data=True)):
                        category_id = G.nodes[v].get('categoryID')
                        recommended_categories[category_id] = recommended_categories.get(category_id, 0) + 1

        # Sort and get top 2 categories
        sorted_categories = sorted(recommended_categories, key=recommended_categories.get, reverse=True)[:2]

    else:
        # If no similar users, pick 2 random categories from the same supercategory
        potential_categories = [u for u, v, d in G.edges(key_category_node, data=True) 
                                if d.get('label') == 'CATEGORY_BELONGS_TO' and G.nodes[u].get('categoryID') != key_category_id]
        # no duplicates
        potential_categories = list(set(potential_categories))

        # Randomly select up to 2 categories
        selected_nodes = random.sample(potential_categories, 2) 
        sorted_categories = [G.nodes[v].get('categoryID') for v in selected_nodes]

    return [int(x) for x in sorted_categories[:2]]


In [46]:
userID = 100
key_category = 238482
video_subject = -1

In [47]:
tab2_cat, tab3_cat = category_recommender(G, userID, key_category)
tab2_cat, tab3_cat
# userID 중에 10000000은 없지만, 이 경우에는 similar user가 없는 것으로 된다.

(225107, 275980)

# Item recommendation

In [48]:
def item_recommender(G, user_id, category_id):
    # Step 1: Find similar users and their rated items
    target_user_node = find_node_by_attribute(G, 'userID', str(user_id))
    if target_user_node:
        similar_users = [v for u, v in G.edges(target_user_node) if G[u][v].get('label') == 'SIMILAR']
    else:
        similar_users = []
    item_scores = {}

    print(similar_users)
    # items that target_user already bought
    target_user_items = [v for u, v, d in G.edges(target_user_node, data=True) if d.get('label') == 'RATED']
    for su in similar_users:
        print(type(su))
        for u, item, data in G.edges(data=True):
            if data.get('label') == 'RATED' and item not in target_user_items and u == su:
                item_data = G.nodes[item]
                # 하나의 item이 동시에 여러 category에 속할 수 있으므로 (['공통']의 경우), 그것을 반영합니다
                category_ids = [G.nodes[v].get('categoryID') for u, v, d in G.edges(item, data=True) if d.get('label') == 'BELONGS_TO']
                if str(category_id) in category_ids:
                    # item 평점을 가져옵니다. 이로 우선순위를 부여합니다.
                    rating = data.get('Rating', 0)
                    item_scores[item] = item_scores.get(item, 0) + float(rating)
        print(item_scores)
        print('category_ids:')
        print(category_ids)
    # Step 2: item을 rating을 기준으로 높은 것을 추천해줍니다.
    top_items = sorted(item_scores, key=item_scores.get, reverse=True)[:4]
    print(top_items)
    # Step 3: Cold Start Problem
    # similar user가 없는 경우, 또는 추천하는 item이 부족한 경우, 자체적인 score를 계산하여 추천을 합니다.
    # 동일 category 내에서 추천을 해줍니다.
    # Calculate the number of recommendations already found
    num_of_recommendations = len(top_items)
    num_of_needed_recommendations = 4 - num_of_recommendations
    print(num_of_needed_recommendations)

    # Cold Start Problem Handling
    if num_of_needed_recommendations > 0:
         # Find the node that represents the specified category
        category_node = find_node_by_attribute(G, 'categoryID', str(category_id))
    
        # Find all item nodes connected to this category node and not in top_items
        all_items_in_category = []
        if category_node:m
            
            for u, v, d in G.edges(data=True):
                #print("Edge from", u, "to", v, "with data:", d)
                if d.get('label') == 'BELONGS_TO' and v == category_node and u not in top_items:
                    all_items_in_category.append(u)
    
        # Calculate a score for each additional item based on NumOfReviews and Rating
        cold_start_scores = {item: float(G.nodes[item].get('NumOfReviews', 0)) * float(G.nodes[item].get('Rating', 0)) for item in all_items_in_category}
        # Select additional items based on the score
        additional_items = sorted(cold_start_scores, key=cold_start_scores.get, reverse=True)[:num_of_needed_recommendations]
        top_items.extend(additional_items)
    
    # Convert node IDs to itemIDs
    recommended_itemIDs = [(G.nodes[item].get('itemID')) for item in top_items]

    return recommended_itemIDs

In [49]:
tab2_item = item_recommender(G, userID, tab2_cat)
tab3_item = item_recommender(G, userID, tab3_cat)
tab2_item, tab3_item

([41715, 41542, 41600, 41783], [14438, 14640, 14678, 14503])

# Random recommendation

In [50]:
def random_recommender(G, tab2_cat, tab3_cat, video_subject):
    # Find all supercategories belonging to the given metacategory
    # Find the metacategory node
    metacategory_node = None
    for node, data in G.nodes(data=True):
        if data.get('metacategoryID') == str(video_subject):
            metacategory_node = node
    
    # Find all supercategories under this metacategory
    supercategories = [u for u, v, d in G.edges(data=True) if d.get('label') == 'SUPERCATEGORY_BELONGS_TO' and v == metacategory_node]

    # Find all categories within these supercategories, excluding tab2_cat and tab3_cat
    potential_categories = []
    for sc in supercategories:
        categories = [u for u, v, d in G.edges(data=True) if d.get('label') == 'CATEGORY_BELONGS_TO' and v == sc]
        potential_categories.extend(categories)

    # Filter categories and randomly select one
    potential_categories = [cat for cat in potential_categories if G.nodes[cat].get('categoryID') not in [tab2_cat, tab3_cat]]
    tab4_cat = random.choice(potential_categories) if potential_categories else None
    
    # Randomly recommend items from the selected category
    tab4_items = []
    if tab4_cat:
        items_in_category = [u for u, v, d in G.edges(data=True) if d.get('label') == 'BELONGS_TO' and v == str(tab4_cat)]
        tab4_items = random.sample(items_in_category, min(len(items_in_category), 4))

    # Convert node IDs to itemIDs
    tab4_item_ids = [int(G.nodes[item].get('itemID')) for item in tab4_items]

    return int(G.nodes[tab4_cat].get('categoryID')), tab4_item_ids


In [51]:
random_recommender(G, tab2_cat, tab3_cat, -1)

(381526, [44131, 44192, 43921, 44133])